# MLP

In [5]:
import numpy as np


In [6]:
class PerceptronMulticapa:
    def __init__(self, capas, alpha=0.1):
        self.capas = capas
        self.alpha = alpha
        self.bias = []
        self.pesos = []
        for i in range(0, len(capas) - 1):
            # Inicializar los pesos y bias de cada capa
            peso = np.random.randn(capas[i], capas[i+1])
            self.pesos.append(peso)
            bias = np.random.randn(capas[i+1])
            self.bias.append(bias)

    def activacion(self, x):
        # Función de activación sigmoide
        return 1.0 / (1 + np.exp(-x))

    def activacion_derivada(self, x):
        # Derivada de la función de activación sigmoide
        return x * (1 - x)

    def feedforward(self, X):
        # Calcular la salida de cada capa
        capa_activacion = [X]
        for i in range(0, len(self.capas) - 1):
            x = np.dot(capa_activacion[i], self.pesos[i]) + self.bias[i]
            y = self.activacion(x)
            capa_activacion.append(y)
        return capa_activacion

    def backpropagation(self, X, y, capa_activacion):
        # Calcular el error de la capa de salida
        error = capa_activacion[-1] - y
        delta = error * self.activacion_derivada(capa_activacion[-1])
        
        # Propagar el error hacia atrás a través de la red neuronal
        for i in reversed(range(0, len(self.capas) - 1)):
            activacion_actual = capa_activacion[i]
            activacion_anterior = capa_activacion[i-1] if i > 0 else X
            d_peso = np.outer(activacion_anterior, delta)
            d_bias = delta
            self.pesos[i] -= self.alpha * d_peso
            self.bias[i] -= self.alpha * d_bias
            delta = np.dot(delta, self.pesos[i].T) * self.activacion_derivada(activacion_actual)

    def entrenar(self, X, y, epochs):
        for epoch in range(0, epochs):
            for i in range(0, len(X)):
                # Feedforward
                capa_activacion = self.feedforward(X[i])

                # Backpropagation
                self.backpropagation(X[i], y[i], capa_activacion)

    def predecir(self, X):
        # Obtener la salida de la última capa
        capa_activacion = self.feedforward(X)
        return capa_activacion[-1]
    

In [22]:
import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
import pandas as pd
# Cargar el conjunto de datos Iris

iris = load_iris()
X = iris.data
y = iris.target

# Dividir el conjunto de datos en entrenamiento y prueba
X_entrenamiento, X_prueba, y_entrenamiento, y_prueba = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)



In [23]:
mu = np.mean(X, 0)
sigma = np.std(X, 0)
X = (X - mu ) / sigma

In [24]:
# Crear y entrenar el perceptrón multicapa
perceptron = PerceptronMulticapa(capas=[4,4,4,3], alpha=0.225)
perceptron.entrenar(X_entrenamiento, np.eye(3)[y_entrenamiento], epochs=1000)

# Hacer predicciones sobre el conjunto de prueba
predicciones = []
for i in range(len(X_prueba)):
    prediccion = perceptron.predecir(X_prueba[i])
    prediccion_clase = np.argmax(prediccion)
    predicciones.append(prediccion_clase)

# Calcular la precisión de las predicciones
print("Predicciones: ", predicciones)
print("Y_Real      : ", y_prueba)
precision = sum(predicciones == y_prueba) / len(y_prueba)
print(f"Precisión: {precision}")

Predicciones:  [0, 2, 1, 1, 0, 1, 0, 0, 2, 1, 2, 2, 2, 1, 0, 0, 0, 1, 1, 2, 0, 2, 1, 2, 2, 1, 1, 0, 2, 0]
Y_Real      :  [0 2 1 1 0 1 0 0 2 1 2 2 2 1 0 0 0 1 1 2 0 2 1 2 2 1 1 0 2 0]
Precisión: 1.0


# MLP con Sklearn

In [6]:
from sklearn.neural_network import MLPClassifier
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

In [7]:
mlp = MLPClassifier(hidden_layer_sizes=(10, 10), activation= 'tanh', max_iter=10000, random_state=42, solver="sgd")
mlp.fit(X_entrenamiento, y_entrenamiento)

y_pred = mlp.predict(X_prueba)

print("Predicciones: ", y_pred)
print("Valor real  : ",y_prueba)

accuracy = accuracy_score(y_prueba, y_pred)
print("Precisión: {:.2f}".format(accuracy))

confusion_matrix(y_prueba,y_pred)

Predicciones:  [0 2 1 1 0 1 0 0 2 1 2 2 2 1 0 0 0 1 1 2 0 2 1 2 2 1 1 0 2 0]
Valor real  :  [0 2 1 1 0 1 0 0 2 1 2 2 2 1 0 0 0 1 1 2 0 2 1 2 2 1 1 0 2 0]
Precisión: 1.00


array([[10,  0,  0],
       [ 0, 10,  0],
       [ 0,  0, 10]], dtype=int64)